Extract

From the original dataset, we need to extract the country codes (from server_request_country_code), the total number of visits (based on number of repetitions from each country code) and then filter based on a date range (WHERE).

In [ ]:
import boto3
import os
import time
import psycopg2

access_key = os.environ.get('AWS_ACCESS_KEY_ID')
secret_key = os.environ.get('AWS_SECRET_ACCESS_KEY')

# Set up the Athena client
athena_client = boto3.client(
    'athena',
    region_name='eu-west-2',
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key)

# Write the SQL query
sql_query = """
    SELECT server_request_country_code, COUNT(*) as total_visits
    FROM vod_clickstream
    WHERE datetime >= CAST('2018-01-01' AS timestamp) AND datetime < CAST('2018-02-01' AS timestamp)
    GROUP BY server_request_country_code;
"""


# Execute the Athena query
query_execution = athena_client.start_query_execution(
    QueryString=sql_query,
    QueryExecutionContext={
        "Database": "ilhaam-athena-parquet"
    },
    ResultConfiguration={
        "OutputLocation": "s3://athena-learners-etl-bite05/ilhaam/" # <= This will be different for you, refer to the Amazon Athena pill for more information.
    }
)

# Poll the query status until it is either successful or failed
query_status = "QUEUED"
query_execution_id = query_execution["QueryExecutionId"]

while query_status in ["QUEUED", "RUNNING"]:
    query_execution = athena_client.get_query_execution(
        QueryExecutionId=query_execution_id
    )
    query_status = query_execution["QueryExecution"]["Status"]["State"]
    if query_status == "FAILED":
        raise Exception("Athena query failed!")
    time.sleep(1)

# Retrieve the query results
results = athena_client.get_query_results(
    QueryExecutionId=query_execution_id
)["ResultSet"]["Rows"]

In [3]:
# Connect to the local Postgres database
conn = psycopg2.connect(database="etl_bites", 
                        user="ilhaam.ahmed", 
                        password="etl_proj", 
                        host="localhost", 
                        port="5432")
cursor = conn.cursor()

# Create the table if it doesn't exist
cursor.execute("""
    CREATE TABLE IF NOT EXISTS visits_per_country (
        country_code VARCHAR(2) PRIMARY KEY,
        total_visits INTEGER
    );
""")
            
# Process the query results
for row in results[1:]:
    if not row["Data"][0] or not row["Data"][1]:
        # You could also print what a `row` has if you are curious!
        print(f"Skipping row: {row}")
        continue

    country_code = row["Data"][0]["VarCharValue"]
    total_visits = int(row["Data"][1]["VarCharValue"])

    # Insert the data into the local PostgreSQL database
    insert_query = """
        INSERT INTO visits_per_country (country_code, total_visits)
        VALUES (%s, %s)
        ON CONFLICT (country_code)
        DO UPDATE SET total_visits = EXCLUDED.total_visits;
    """

    try:
        cursor.execute(insert_query, (country_code, total_visits))
        
    except Exception as e:
        print("Error occurred inserting into analytical DB: %s"% e)
        conn.rollback()  # Rollback the transaction if there's an error

# Commit the changes and close the cursor and connection outside the loop
conn.commit()
cursor.close()
conn.close()